In [1]:
import pandas as pd
import numpy as np
import pickle
from train_score import Ranking
import os
import openpyxl
FNAME = '押し引きシミュレーター.xlsx'
wb=openpyxl.load_workbook(FNAME)

In [2]:
p = pickle.load(open('score_predictor.bin', 'rb'))
f = np.array([25000, 26000, 24000, 25000])
r = Ranking(p)
ret = r.predict(1, f)
print(ret)

[[0.21981544 0.2726554  0.2651835  0.24234566]
 [0.27375166 0.26413387 0.2420128  0.22010168]
 [0.22006526 0.24453762 0.25488844 0.28050868]
 [0.28636765 0.21867311 0.23791526 0.25704398]]


In [3]:
def main():
    result, point = calc_rank_probability([24000, 25000, 25000, 25000], 1000, 30, 2)
    display(result)
    display(point)

    # シートへの書き込み
    sheet = wb['入力項目']
    ROW_START = 17
    COL_START = 2
    for row in range(6):
        # 4人の素点
        for col in range(4):
            sheet.cell(ROW_START + row, COL_START + col).value = point[roｗ][col]
        # 各順位の確率
        for col in range(4):
            sheet.cell(ROW_START + row, COL_START + 5 + col).value = result[roｗ][col]
    
    wb.save(FNAME)

In [4]:
# P1を親、P2（自分）を子想定

def calc_rank_probability(point_list, deposit, hu, han):
    """
    point_list: 4人の点数
    deposit: 供託
    hu: 自分アガリ時の符
    han: 自分アガリ時の翻数
    """
    p = pickle.load(open('score_predictor.bin', 'rb'))
    r = Ranking(p)
    
#     自分をP2、相手(親)をP1とする
    P1 = point_list[0]
    P2 = point_list[1]
    P3 = point_list[2]
    P4 = point_list[3]
    DEPOSIT = deposit
    
    result = []
    point = []
    
#     自分の点数を求める
    tensu_df = pd.read_csv('点数表.csv')
    df = tensu_df[(tensu_df['符'] == hu) & (tensu_df['翻数'] == han)]
    display(df)
    
    # P2のツモ
    p1 = P1 - df['子ツモ親'].values[0]
    p2 = P2 + df['子ツモ親'].values[0] + df['子ツモ子'].values[0] * 2 + DEPOSIT
    p3 = P3 - df['子ツモ子'].values[0]
    p4 = P4 - df['子ツモ子'].values[0]
    
    f = np.array([p1, p2, p3, p4])
    ret = r.predict(1, f)
#     print(ret)
    result.append(ret[1])
    point.append([p1, p2, p3, p4])
    
    # P2のロン
    p1 = P1 - df['子ロン'].values[0]
    p2 = P2 +  df['子ロン'].values[0]  + DEPOSIT
    p3 = P3
    p4 = P4
    
    f = np.array([p1, p2, p3, p4])
    ret = r.predict(1, f)
#     print(ret)
    result.append(ret[1])
    point.append([p1, p2, p3, p4])
    
    
    # P1のツモ
#     平均打点は一発なしで10000点なので30符4ハンで計算する
    p1 = P1 + 12000 + DEPOSIT
    p2 = P2 - 4000
    p3 = P3 - 4000
    p4 = P4 - 4000

    f = np.array([p1, p2, p3, p4])
    ret = r.predict(0, f)
    result.append(ret[1])
    point.append([p1, p2, p3, p4])
    
    
    # P1のロン
#     平均打点は一発なしで7500点なので40符3ハンで計算する
    p1 = P1 + 7700 + DEPOSIT
    p2 = P2 - 7700
    p3 = P3
    p4 = P4

    f = np.array([p1, p2, p3, p4])
    ret = r.predict(0, f)
    result.append(ret[1])
    point.append([p1, p2, p3, p4])
    
    
    # テンパイ流局
    p1 = P1+ 1500
    p2 = P2 + 1500
    p3 = P3 -  1500
    p4 = P4 - 1500
    f = np.array([p1, p2, p3, p4])
    ret = r.predict(0, f)
    result.append(ret[1])
    point.append([p1, p2, p3, p4])
    
    
    # ノーテン流局
    p1 = P1 + 3000
    p2 = P2 - 1000
    p3 = P3 -  1000
    p4 = P4 - 1000
    f = np.array([p1, p2, p3, p4])
    ret = r.predict(0, f)
    result.append(ret[1])
    point.append([p1, p2, p3, p4])
    
    
#     display(result)
    return result, point

In [5]:
main()

,符,翻数,親ツモ,親ロン,子ツモ親,子ツモ子,子ロン
7,30,2,1000.0,2900.0,1000.0,500.0,2000.0


[array([0.34421577, 0.28294327, 0.21346344, 0.15937752]),
 array([0.34226312, 0.28597121, 0.21407824, 0.15768742]),
 array([0.13638239, 0.25251932, 0.30497661, 0.30612169]),
 array([0.11285384, 0.17060429, 0.24084924, 0.47569264]),
 array([0.2704659 , 0.26325541, 0.26187188, 0.20440681]),
 array([0.22816185, 0.24415774, 0.26399055, 0.26368986])]

[[23000.0, 28000.0, 24500.0, 24500.0],
 [22000.0, 28000.0, 25000, 25000],
 [37000, 21000, 21000, 21000],
 [32700, 17300, 25000, 25000],
 [25500, 26500, 23500, 23500],
 [27000, 24000, 24000, 24000]]